# Image Recommendation System via Collaborative Filtering

# ***Please read the instructions very carefully***

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   ⚠️ The Assignment will be evaluated automatically. Please adhere to taking proper inputs from `config.csv` file. You can change your `config.csv` file to experiment with your code. But at the end, make sure that your outputs are corresponding to input values in `config.csv`
5.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

## About the Dataset
Behance is a community art website where users showcase and discover creative work. Each user is able to “appreciate” (equivalent to a “like” on Instagram or a “react” on Facebook) an image, indicating that they like the image. It is in the website’s best interests to show users pictures that they would like, to keep them engaged for longer. For this question, given a set of pictures that a user has already appreciated, you have to show them a new picture that they would like based on what similar users appreciated.


<br><br>
**The dataset has information of 1 million appreciates of 63,497 users on 178,788 items. The file Behance appreciate 1M has a triplet in each line in the form of (user id, item id, unix timestamp).**

**Task: Take the inputs from the config.csv file and output the recommendations for a particular person**
- Collaborative Filtering is a way to predict items to the user based on the the
user’s history and the history of similar users. The similarity between users can be quantified by the number of images that both the users appreciated.
- The images appreciated by a similar user would be the most suitable images to show a user. Since we can find the similarity between any two users, we would be able to find the “nearest” neighbours of any user, allowing us to use a KNN-based algorithm to recommend new images to a user.
- Since people do not like seeing pictures that they have seen already. Make sure that you do not recommend pictures that a user has appreciated already.
- Output the final response will be saved in the file named ```config['output_file']```.


**Output file format:**
Populate the output file with images that the user has not seen of the k most
similar users, in descending order of their similarity. Each line in the output
file should be a duplet in the form of (item id, user id), where the user id is the
id of the kth similar user. The order of the images corresponding to the same
similar user would not matter. The output file would look something like this:
```
item_id_1_of_1st_similar_user 1st_most_similar_user_id
item_id_2_of_1st_similar_user 1st_most_similar_user_id
item_id_3_of_1st_similar_user 1st_most_similar_user_id
...
item_id_1_of_2nd_similar_user 2nd_most_similar_user_id
item_id_2_of_2nd_similar_user 2nd_most_similar_user_id
item_id_3_of_2nd_similar_user 2nd_most_similar_user_id
...
item_id_1_of_kth_similar_user kth_most_similar_user_id
item_id_2_of_kth_similar_user kth_most_similar_user_id
item_id_3_of_kth_similar_user kth_most_similar_user_id
```

The dataset was extracted using Behance’s API as a part of the paper
“Vista: A visually, socially, and temporally-aware model for artistic
recommendation, RecSys, 2016”. Check out this [Google Drive folder](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA) for
more information about the dataset.


Have fun! The users are waiting to see new pictures!

### Import necessary libraries

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Config Generation Sample Code.
# ⚠️ Only for experimentation on your side.
# ⚠️ Should be commented during submission.

# df = pd.DataFrame(data=[{'id':10356,
#                   'k':5,
#                   'dataset_file':'./Behance_subsampled.txt',
#                   'output_file':'./output.txt'}])
# df.to_csv('config.csv')

In [3]:
config = pd.read_csv('config.csv').iloc[0]

In [4]:
config

Unnamed: 0                             0
id                                 10356
k                                      5
dataset_file    ./Behance_subsampled.txt
output_file                 ./output.txt
Name: 0, dtype: object

In [5]:
user = config['id']
k_value = config['k']

### Read the Data

In [6]:
with open(config['dataset_file'], 'r') as inFile:
    appreciate_data = inFile.readlines()

In [7]:
# your code here

### Initialize a dictionary to store the item_ids that a user likes

### Go through each line of the input file and construct the user_likes dictionary

In [7]:
user_likes = dict()
items = set()
users = set()

In [8]:
for line in appreciate_data:
    line = line.strip()
    
    user_id = int(line.split()[0])
    item_id = int(line.split()[1])

    items.add(item_id)
    users.add(user_id)

    if user_id not in user_likes:
        user_likes[user_id] = list()
    
    user_likes[user_id].append(item_id)

In [9]:
items = sorted(items)
items[0]
users = sorted(users)
users[0]
# users

10356

In [10]:
user_likes_matrix = [ [0 for i in range(len(items))] for i in range(len(list(user_likes.keys()))) ]

In [11]:
for user_idx in range(len(user_likes.keys())):
    for item_idx in range(len(user_likes_matrix[user_idx])):
        if items[item_idx] in user_likes[users[user_idx]]:
            user_likes_matrix[user_idx][item_idx] = 1

In [12]:
# getting latent features
import numpy as np
from sklearn.decomposition import TruncatedSVD

def get_latent_features(user_likes_matrix):
    epsilon = 1e-9
    n_latent_factors = 1000
    user_svd = TruncatedSVD(n_components = n_latent_factors)
    user_features = user_svd.fit_transform(user_likes_matrix) + epsilon

    return user_features

In [13]:
user_features = get_latent_features(user_likes_matrix).astype("float32")
type(user_features)

numpy.ndarray

In [20]:
len(user_features[0])

1000

### Use KNN after Collaborative Filtering to find nearest neighbors

In [14]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

def neighbors(user,user_features,k_value):
    """ returns an iterable object (like list or generator) of top k similar users """
    # run through every user and calculate number of same posts liked
    # keep k (similarity, user_id) as a list and sort each iteration if list > k
    # each iteration compare with smallest similarity score in the list
    top_k = []
    
    given_user_idx = -1
    for user_idx in range(len(user_features)):
        if users[user_idx] == user:
            given_user_idx = user_idx
            break
    given_user_feature = user_features[given_user_idx]

    for user_idx,user_feature in enumerate(user_features):
        if users[user_idx] == user: continue
        
        vec1 = np.array([user_features[user_idx]])
        vec2 = np.array([given_user_feature])

        similarity = euclidean_distances(vec1,vec2)

        if len(top_k) < k_value: top_k.append((similarity,users[user_idx]))
        else:
            if similarity < top_k[k_value-1][0]:
                top_k[k_value-1] = (similarity,users[user_idx])
                top_k = sorted(top_k)
    
    top_k_users = [t[1] for t in top_k]

    return top_k_users

In [16]:
# this function is only called to check if top k returned by using matrix factorization is same as while using jaccard - USED ONLY TO PERSONALLY CHECK IF ANSWERS MAKE SENSE NOT USED FOR ASSIGNMENT QUESTION PURPOSES 
def jaccard_neighbors(user,k_value):
    """ returns an iterable object (like list or generator) of top k similar users """
    # run through every user and calculate number of same posts liked
    # keep k (similarity, user_id) as a list and sort each iteration if list > k
    # each iteration compare with smallest similarity score in the list
    top_k = []
    for user_id in list(user_likes.keys()):
        if user_id == user: continue

        similarity = len(list(set(user_likes[user]).intersection(user_likes[user_id]))) / len(list(set(user_likes[user]).union(user_likes[user_id])))

        if len(top_k) < k_value: top_k.append((similarity,user_id))
        else:
            if similarity > top_k[k_value-1][0]:
                top_k[k_value-1] = (similarity,user_id)
                top_k = sorted(top_k,reverse=True)
    
    top_k_users = [t[1] for t in top_k]

    return top_k_users

### Open the output file to write all the lines to the file

In [17]:
outFile = open(config['output_file'], 'w')

for n_user in neighbors(user,user_features,k_value):
    user_id = n_user
    for item_id in user_likes[user_id]:
        if item_id not in user_likes[user]:
            outFile.write(str(item_id) + ' ' + str(user_id) + '\n')

outFile.close()

[15341, 18197, 19830, 28270, 32901]
[104300, 206299, 333388, 366769, 366885]
